In [1]:
import os
import json
from sklearn.model_selection import train_test_split
import shutil

In [2]:
base_dir = r'D:\lakemapping\U_Net'
dataset_dir=r'D:\lakemapping\2_dataset\sample747'
# base_dir = r'/media/nkd/backup/5_lakemapping/U_Net'
# dataset_dir=r'/media/nkd/backup/5_lakemapping/sample600'
image_type = '.png'       
NDWI_fn = 'ndwi'
red_fn = 'red'
blue_fn = 'blue'
green_fn = 'green'
swir_fn = 'swir'
annotation_fn = 'annotation'
type_num=6
patch_size = (512,512,6) 
patch_dir = os.path.join(base_dir,'patches{}'.format(patch_size[0])) 
# The training areas are divided into training, validation and testing set. Note that training area can have different sizes, so it doesn't guarantee that the final generated patches (when using sequential stratergy) will be in the same ratio.
test_ratio = 0.2
val_ratio = 0.25

## 划分数据集
从output文件夹存储到patchReshape文件夹，train和test按类别type分别存储

In [ ]:
def moveFileinType(lakeType,patchType,fn):
    output_dir=os.path.join(dataset_dir,r'patchesReshape/{}/type{}'.format(patchType,lakeType))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    move(output_dir,fn)
    move(output_dir,fn.replace(NDWI_fn,annotation_fn))
    move(output_dir,fn.replace(NDWI_fn,red_fn))
    move(output_dir,fn.replace(NDWI_fn,green_fn))
    move(output_dir,fn.replace(NDWI_fn,blue_fn))
    move(output_dir,fn.replace(NDWI_fn,swir_fn))
    
def move(output_dir,fn):
    new=os.path.join(output_dir,fn)
    old=os.path.join(patch_output,fn)
    shutil.move(old, new)

In [ ]:
# split dataset and copy images & annotations to new directory
for i in range(0,type_num):
    frames = []
    frames_json = os.path.join(patch_dir,'frames_list{}.json'.format(i))
    patch_output = os.path.join(dataset_dir,'output/output{}'.format(i))
    all_files = os.listdir(patch_output)
    all_files_NDWI = [fn for fn in all_files if fn.startswith(NDWI_fn) and fn.endswith(image_type)]#ndwi.png
    len(all_files_NDWI)
    print(all_files_NDWI)

    if os.path.isfile(frames_json):
        print("dataset type{} had been splited".format(i))
#             train data are classified into different dirs for following process.
        with open(frames_json, 'r') as file:
            fjson = json.load(file)#
            for train_fn in fjson['training_frames']:
                moveFileinType(i,'train',train_fn)
            for testing_fn in fjson['testing_frames']:
                moveFileinType(i,'test',testing_fn)
            for validation_fn in fjson['validation_frames']:
                moveFileinType(i,'val',validation_fn)
    else:
            print("Creating and writing train-test split from file")
            frames_list = list(range(len(all_files_NDWI)))
            
            # Divide into training and test set       
            training_frames, testing_frames = train_test_split(frames_list, test_size=test_ratio)
            # Further divide into training set into training and validataion set              
            training_frames, validation_frames = train_test_split(training_frames, test_size=val_ratio)
            
            training_frames_name=[all_files_NDWI[id] for id in training_frames]
            testing_frames_name=[all_files_NDWI[id] for id in testing_frames]
            validation_frames_name=[all_files_NDWI[id] for id in validation_frames]
            
            #train data are classified into different dirs for following process.
#             for train_fn in training_frames_name:
#                 moveFileinType(i,'train',train_fn)
#             for testing_fn in testing_frames_name:
#                 moveFileinType(i,'test',testing_fn) 
#             for validation_fn in validation_frames_name:
#                 moveFileinType(i,'val',validation_fn)
                
            frame_split = {
                'training_frames': training_frames_name,
                'testing_frames': testing_frames_name,
                'validation_frames': validation_frames_name
            }
            
            if not os.path.exists(patch_dir):
                os.makedirs(patch_dir)
            with open(frames_json, 'w') as f:
                json.dump(frame_split, f)
                
            print('training_frames', training_frames_name)
            print('validation_frames',validation_frames_name )
            print('testing_frames', testing_frames_name)